In [ ]:
prune_all_forums
# delete old topics of all forums

In [ ]:
prune_forum
# delete old topic of a single forum

In [ ]:
prune_notifications
#Prune notifications cron task.

# notifications_table is forum related
$sql = 'DELETE FROM ' . $this->notifications_table . '
			WHERE notification_time < ' . (int) $timestamp .
			(($only_read) ? ' AND notification_read = 1' : '');

In [ ]:
prune_shadow_topics
# Prune one forum of its shadow topics cron task.


$sql = 'SELECT forum_name
			FROM ' . FORUMS_TABLE . "
			WHERE forum_id = $forum_id";
$result = $this->db->sql_query($sql, 3600);
$row = $this->db->sql_fetchrow($result);
$this->db->sql_freeresult($result);

if ($row)
{
    $prune_date = time() - ($prune_days * 86400);
    $next_prune = time() + ($prune_freq * 86400);

    prune($forum_id, $prune_mode, $prune_date, $prune_flags, true);

    $sql = 'UPDATE ' . FORUMS_TABLE . "
        SET prune_shadow_next = $next_prune
        WHERE forum_id = $forum_id";
    $this->db->sql_query($sql);

    $user_id = (empty($this->user->data)) ? ANONYMOUS : $this->user->data['user_id'];
    $user_ip = (empty($this->user->ip)) ? '' : $this->user->ip;

    $this->log->add('admin', $user_id, $user_ip, 'LOG_PRUNE_SHADOW', false, array($row['forum_name']));
}

In [ ]:
queue
# email and jabber queue

In [ ]:
tidy_cache
# have no idea what is auto GC, this is forum related.


global $config;

# cache has auto GC, no need to have any code here :)
$config->set('cache_last_gc', time(), false);


In [ ]:
tidy_database
# forum id related


global $config, $db;

# Here we check permission consistency

# Sometimes, it can happen permission tables having forums listed which do not exist
$sql = 'SELECT forum_id
    FROM ' . FORUMS_TABLE;
$result = $db->sql_query($sql);

$forum_ids = array(0);
while ($row = $db->sql_fetchrow($result))
{
    $forum_ids[] = $row['forum_id'];
}
$db->sql_freeresult($result);

# Delete those rows from the acl tables not having listed the forums above
$sql = 'DELETE FROM ' . ACL_GROUPS_TABLE . '
    WHERE ' . $db->sql_in_set('forum_id', $forum_ids, true);
$db->sql_query($sql);

$sql = 'DELETE FROM ' . ACL_USERS_TABLE . '
    WHERE ' . $db->sql_in_set('forum_id', $forum_ids, true);
$db->sql_query($sql);

$config->set('database_last_gc', time(), false);

In [ ]:
tidy_plupload
# clear old upload images, dont really know what is this about

In [ ]:
tidy_search
# empty

In [ ]:
tidy_sessions
# forum and user related, not a good idea to use as system cron


('session_gc','3600',0), 


function session_gc()

$sql = 'DELETE FROM ' . SESSIONS_TABLE . '
			WHERE session_user_id = ' . ANONYMOUS . '
				AND session_time < ' . (int) ($this->time_now - $config['session_length']);
        
while
$sql = 'DELETE FROM ' . SESSIONS_TABLE . '
				WHERE ' . $db->sql_in_set('session_user_id', $del_user_id) . '
					AND session_time < ' . ($this->time_now - $config['session_length']);
        
$sql = 'DELETE FROM ' . SESSIONS_KEYS_TABLE . '
					WHERE last_login < ' . (time() - (86400 * (int) $config['max_autologin_time']));
				$db->sql_query($sql);
                
$sql = 'DELETE FROM ' . LOGIN_ATTEMPT_TABLE . '
				WHERE attempt_time < ' . (time() - (int) $config['ip_login_limit_time']);

In [ ]:
tidy_warnings
# delete warnings of current forum


('warnings_gc','14400',0), 


global $db, $config;

$expire_date = time() - ($config['warnings_expire_days'] * 86400);
$warning_list = $user_list = array();

$sql = 'SELECT * FROM ' . WARNINGS_TABLE . "
    WHERE warning_time < $expire_date";
$result = $db->sql_query($sql);

while ($row = $db->sql_fetchrow($result))
{
    $warning_list[] = $row['warning_id'];
    $user_list[$row['user_id']] = isset($user_list[$row['user_id']]) ? ++$user_list[$row['user_id']] : 1;
}
$db->sql_freeresult($result);

if (sizeof($warning_list))
{
    $db->sql_transaction('begin');

    $sql = 'DELETE FROM ' . WARNINGS_TABLE . '
        WHERE ' . $db->sql_in_set('warning_id', $warning_list);
    $db->sql_query($sql);

    foreach ($user_list as $user_id => $value)
    {
        $sql = 'UPDATE ' . USERS_TABLE . " SET user_warnings = user_warnings - $value
            WHERE user_id = $user_id";
        $db->sql_query($sql);
    }

    $db->sql_transaction('commit');
}

$config->set('warnings_last_gc', time(), false);